In [1]:
import sys
sys.path.append(r"d:\gis\toolboxes\pysda")

In [2]:
# import pysda modules
import sdapoly, sdaprop, sdainterp

In [3]:
# get/set an aoi
myshp = r"F:\ZBOOK\GIS\TEMP\fs.shp"
myaoi = sdapoly.shp(myshp)

In [ ]:
# tell matplot lib to publish here
%matplotlib inline

# visualize
myaoi.plot()

In [4]:
# what ssurgo properties can we get?  Three functions tell us.
# (numeric) numprops, (categorical)catprops, allprops
# aggregation method selected guides which properties are available.

# aggregation methods: weighted avergae (wtd_avg),  dominant component numeric (dom_comp_num), minimum maximum (minmax)
# can get numeric rpoperties (percent clay, unrubber fiber percent, cation exchange capacity,...)

# aggregation methods: dominant component categorical (dom_comp_cat), dominant condition (dom_cond)
# can get categorical properties (hydrologic soil group, drainage class, taxonomic order,...)

# use column name where assigning proprerty for sdaprop.getprop()
# to see the properties

# sdaprop.numprops()
sdaprop.catprops()


{'Corrosion of Concrete': 'corcon',
 'Corrosion of Steel': 'corsteel',
 'Drainage Class': 'drainagecl',
 'Hydrologic Group': 'hydgrp',
 'Taxonomic Class Name': 'taxclname',
 'Taxonomic Order': 'taxorder',
 'Taxonomic Suborder': 'taxsuborder',
 'Taxonomic Temperature Regime': 'taxtempregime',
 'Wind Erodibility Group': 'weg',
 'Wind Erodibility Index': 'wei',
 't Factor': 'tfact'}

In [5]:
# now to get SSURGO property, sandtotal_r
wtdavg=sdaprop.getprop(df=myaoi,column='mukey',method='wtd_avg',top=0,bottom=100,prop='sandtotal_r',minmax=None,prnt=False,meta=False)

In [6]:
# remove duplicate columns
myaoi_cols = myaoi.columns.tolist()
wtdavg_cols = wtdavg.columns.tolist()
drop_cols = [col for col in wtdavg_cols if col in myaoi_cols and col != 'mukey']
wtdavg.drop(columns = drop_cols, inplace = True)

# join/merge the results, show first record
mymerge = myaoi.merge(wtdavg, how = 'inner', on = 'mukey')
mymerge.head(1)

,areasymbol,areaname,muname,musym,mukey,nat_musym,geometry,sandtotal_r
0,NC015,"Bertie County, North Carolina",Lynchburg sandy loam,Ly,111444,3qyz,"POLYGON ((-76.81374 36.12652, -76.81422 36.12653, -76.81445 36.12663, -76.81450 36.12666, -76.81483 36.12667, -76.81492 36.12671, -76.81497 36.12678, -76.81499 36.12683, -76.81498 36.12692, -76.81494 36.12699, -76.81460 36.12738, -76.81454 36.12750, -76.81450 36.12772, -76.81439 36.12782, -76.81417 36.12781, -76.81401 36.12779, -76.81388 36.12773, -76.81374 36.12764, -76.81365 36.12752, -76.81350 36.12724, -76.81345 36.12685, -76.81341 36.12679, -76.81336 36.12675, -76.81323 36.12669, -76.81334 36.12663, -76.81349 36.12658, -76.81374 36.12652))",60.00


In [ ]:
# visualize
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1)
fig.set_size_inches(5,5)
mymerge.plot(column = 'sandtotal_r', ax=ax, cmap = 'rainbow',
            legend = True)

# add a legend
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.4,1.0))
ax.set(title = 'Wind Erodibility Group\nDominant Condition')

plt.show()

In [7]:
# mymerge['musym'] = mymerge['musym'].replace('Na', 'Nh')
c = mymerge['musym'].unique().tolist()
remap = dict()
for v in c:
    i = c.index(v) + 1
    remap[v] = i
mymerge['feature.properties.id'] = mymerge['musym'].map(remap)
import pandas as pd
# mymerge['properties.id'] = mymerge['properties.id']
mymerge['sandtotal_r'] = pd.to_numeric(mymerge['sandtotal_r'])


In [10]:
mymerge.head(1)
mymerge.dtypes

areasymbol                 object
areaname                   object
muname                     object
musym                      object
mukey                      object
nat_musym                  object
geometry                 geometry
sandtotal_r               float64
feature.properties.id       int64
dtype: object

In [11]:
import folium
env = mymerge.envelope
cent = env.centroid
x = cent.x[0]
y = cent.y[0]
m = folium.Map([y, x], zoom_start = 10, tiles = 'Stamen Terrain')
# fp = folium.GeoJson(mymerge)
# folium.Choropleth(geo_data = mymerge).add_to(m)
folium.Choropleth(geo_data = mymerge, data = mymerge, columns = ['feature.properties.id','sandtotal_r'], key_on = 'feature.properties.id', fill_color = 'YlGnBu').add_to(m)
m


C:\Program Files\ArcGIS\Pro\bin\Python\envs\pro27_clone\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: key_on `'properties.id'` not found in GeoJSON.

In [ ]:
import pandas as pd
mymerge['sandtotal_r'] = pd.to_numeric(mymerge['sandtotal_r'])

In [ ]:
fp.data.items()

In [ ]:
mymerge

In [ ]:
mymerge.dtypes